# Fine-Tuning a Low-Rank Adaptation (LoRA) Checkpoint Using the 🤗 PEFT Framework and bitsandbytes Library

This notebook presents the process of fine-tuning a Low-Rank Adaptation (LoRA) checkpoint using the 🤗 PEFT (Parameter-Efficient Fine-Tuning) framework and the bitsandbytes library for efficient low-bit quantization.

The objective is to demonstrate how modern parameter-efficient methods can significantly reduce computational and memory requirements during large-language-model fine-tuning while maintaining strong performance.

Key components included:


*   Model Preparation: Load a pretrained transformer-based model and configure it for LoRA fine-tuning.
*   Quantization with bitsandbytes: Apply 8-bit (or 4-bit) quantization to reduce GPU memory usage.
*  Integration with 🤗 PEFT: Implement LoRA adapters so that only targeted parameter updates are required, minimizing the number of trainable weights.
* Training and Evaluation: Run the fine-tuning pipeline on a sample dataset, then evaluate model performance on relevant metrics.
* Performance Analysis: Compare training speed, memory footprint, and accuracy (or loss) between full fine-tuning and the LoRA-based approach.

This notebook serves as a practical and educational resource for researchers and practitioners interested in optimizing large-model fine-tuning using state-of-the-art efficiency techniques.




## Installing Required Libraries

In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 13.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.0/502.0 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 5.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 5.0.0.dev0 which is incompatible.


In [2]:
# Import the `notebook_login` function from the Hugging Face Hub library
# This function allows you to authenticate your notebook session with your Hugging Face account
from huggingface_hub import notebook_login

# Prompt the user to log in to Hugging Face directly within the notebook
# After running this, you'll be asked to enter your Hugging Face API token
# This is required to download models, datasets, or upload your own models to Hugging Face
notebook_login()


## Setup the model

In [3]:
# Import the OS module to interact with the operating system
import os

# Set which GPU device to use (in case of multiple GPUs)
# "0" refers to the first GPU on the system
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Import PyTorch and neural network modules
import torch
import torch.nn as nn

# Import bitsandbytes for memory-efficient 8-bit optimizers and quantization
import bitsandbytes as bnb

# Import Hugging Face Transformers utilities
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-1b7",
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-1b7")

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Explanation / Notes:

* CUDA_VISIBLE_DEVICES controls which GPU(s) are visible to PyTorch. Useful in
multi-GPU setups.

* bitsandbytes (bnb) allows loading very large models in 8-bit, drastically reducing memory usage while maintaining performance.

* AutoModelForCausalLM is used for causal (autoregressive) language models, suitable for text generation tasks.

* AutoTokenizer ensures that the input text is tokenized in the exact same way the model was trained on.

## Freezing the original weights

In [4]:
# Freeze all model parameters to prevent them from being updated during fine-tuning
# We'll only train the adapters (LoRA or PEFT) later
for param in model.parameters():
    param.requires_grad = False  # freeze the model

    # If the parameter is 1-dimensional (e.g., LayerNorm weights or biases)
    # Cast them to float32 for numerical stability
    if param.ndim == 1:
        param.data = param.data.to(torch.float32)

# Enable gradient checkpointing to save memory during training
# This trades compute for memory by re-computing some intermediate activations
model.gradient_checkpointing_enable()

# Allow gradients on the model's input embeddings
# Required for some parameter-efficient fine-tuning methods (like LoRA)
model.enable_input_require_grads()

# Define a wrapper to cast outputs to float32
# This is applied to the final language modeling head (lm_head)
# Helps maintain numerical stability when the model is using mixed precision or 8-bit weights
class CastOutputToFloat(nn.Sequential):
    def forward(self, x):
        return super().forward(x).to(torch.float32)

# Replace the model's lm_head with the CastOutputToFloat wrapper
model.lm_head = CastOutputToFloat(model.lm_head)


Explanation / Notes:

1 ) Freezing Parameters:

* Reduces memory usage and prevents updates to the original weights.

* Only the newly added adapter layers will be trainable.

2 ) Casting 1D Parameters to Float32:

* Small parameters like LayerNorm or biases can be sensitive to low-precision (8-bit) computations.

* Casting them to float32 ensures training stability.

3 ) Gradient Checkpointing:

* Saves memory by storing fewer activations during the forward pass.

* Slightly increases computation because some activations are recomputed during backpropagation.

4 ) enable_input_require_grads():

Some PEFT techniques, like LoRA, require gradients to propagate through input embeddings.

5 ) CastOutputToFloat Wrapper:

* Ensures the model's outputs are in float32 precision, which is important for stable loss computation, especially when using mixed precision or 8-bit quantized models.

## Setting up the LoRa Adapters

In [5]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    Useful for verifying how much of the model is being fine-tuned
    (e.g., after freezing weights and adding LoRA adapters).
    """

    # Initialize counters
    trainable_params = 0  # Parameters that will be updated during training
    all_param = 0         # Total number of parameters in the model

    # Iterate through all model parameters
    for _, param in model.named_parameters():
        all_param += param.numel()  # Count total parameters
        if param.requires_grad:     # Check if parameter is trainable
            trainable_params += param.numel()

    # Print the summary of trainable vs total parameters
    print(
        f"trainable params: {trainable_params} || "
        f"all params: {all_param} || "
        f"trainable%: {100 * trainable_params / all_param:.4f}%"
    )


Explanation:

* Purpose: This function helps confirm that only the LoRA (or PEFT) adapter layers are trainable, while the base model remains frozen.

* param.numel(): Returns the total number of elements (scalars) in a parameter tensor.

* param.requires_grad: Indicates whether the parameter will receive gradients during backpropagation.

* The final print statement reports:

   * Total trainable parameters

   * Total parameters in the model

* Percentage of trainable parameters (a small fraction for LoRA, often <1%)

In [6]:
# Import the necessary modules from the PEFT library
# PEFT (Parameter-Efficient Fine-Tuning) allows adding small, trainable adapter layers
# like LoRA, without updating the entire model.
from peft import LoraConfig, get_peft_model

# Define the LoRA configuration
config = LoraConfig(
    r=16,                # Rank of the LoRA matrices (controls adapter capacity)
    lora_alpha=32,       # Scaling factor to control the contribution of LoRA updates
    # target_modules=["q_proj", "v_proj"],
    # Optionally specify which layers to apply LoRA to (e.g., attention projections).
    # Leaving it commented lets PEFT automatically infer them for supported models.

    lora_dropout=0.05,   # Dropout applied to LoRA layers to prevent overfitting
    bias="none",         # Specifies how biases are handled ("none", "all", or "lora_only")
    task_type="CAUSAL_LM" # Task type; use "CAUSAL_LM" for autoregressive models like BLOOM
)

# Apply the LoRA configuration to the base model
# This injects LoRA adapter layers into the frozen model
# Only these adapter parameters will be trainable
model = get_peft_model(model, config)

# Print how many parameters are now trainable
# Should show that only a small fraction (<1%) of parameters are being updated
print_trainable_parameters(model)


trainable params: 3145728 || all params: 1725554688 || trainable%: 0.1823%


Explanation of Each Part

1) LoraConfig Parameters

* r: Determines the rank of the low-rank matrices. Higher r → more trainable parameters and capacity.

* lora_alpha: A scaling hyperparameter that amplifies LoRA updates before merging them with frozen weights.

* lora_dropout: Helps prevent overfitting by randomly dropping adapter connections during training.

* bias: Defines whether to fine-tune biases as well; "none" freezes all biases.

* task_type: Tells PEFT what kind of model this is — "CAUSAL_LM" for language models, "SEQ_2_SEQ_LM" for encoder-decoder models, etc.

2) get_peft_model()

* Injects adapter layers into the model’s transformer blocks.

* Keeps the main model weights frozen, training only the new LoRA weights.

* This drastically reduces compute and memory needs.

3) print_trainable_parameters(model)

* Confirms that the fine-tuning setup worked.

* Output should show a small percentage of total parameters are trainable.

## Dataset

In [7]:
# Import the Hugging Face Transformers library
# This provides model, tokenizer, and training utilities
import transformers

# Import the Hugging Face Datasets library
# This library offers easy access to thousands of ready-to-use datasets
from datasets import load_dataset

# Load a public dataset of English quotes from the Hugging Face Hub
# The dataset "Abirate/english_quotes" contains short text samples (quotes)
# It will be automatically downloaded and cached the first time you load it
data = load_dataset("Abirate/english_quotes")


README.md: 0.00B [00:00, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

Explanation

1) transformers Library:

* Provides interfaces for pretrained models, tokenizers, and trainers.

* Used here to fine-tune a large language model efficiently with PEFT/LoRA.

2) datasets Library:

* Simplifies loading, preprocessing, and splitting of datasets.

* Integrates seamlessly with transformers.Trainer for model training.

3) Dataset: "Abirate/english_quotes"

* A text dataset containing English-language quotes.

* Each entry is a short sentence, making it suitable for demonstration of fine-tuning causal language models.

In [8]:
# Define a preprocessing function to merge relevant text fields
# This combines the "quote" and its associated "tags" into a single text entry.
# The format used is:  "<quote> ->: <tags>"
def merge_columns(example):
    example["prediction"] = example["quote"] + " ->: " + str(example["tags"])
    return example

# Apply the merge function to each example in the training split
# The Hugging Face `map()` function efficiently applies transformations to all dataset rows
data['train'] = data['train'].map(merge_columns)

# Display the first 5 merged text samples to verify preprocessing
data['train']["prediction"][:5]


Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

["“Be yourself; everyone else is already taken.” ->: ['be-yourself', 'gilbert-perreira', 'honesty', 'inspirational', 'misattributed-oscar-wilde', 'quote-investigator']",
 "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.” ->: ['best', 'life', 'love', 'mistakes', 'out-of-control', 'truth', 'worst']",
 "“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.” ->: ['human-nature', 'humor', 'infinity', 'philosophy', 'science', 'stupidity', 'universe']",
 "“So many books, so little time.” ->: ['books', 'humor']",
 "“A room without books is like a body without a soul.” ->: ['books', 'simile', 'soul']"]

Explanation

1.Purpose of the Function

* The dataset originally contains two separate fields:

   *  "quote" — the main text content (e.g., “Life is beautiful.”)

  *  "tags" — descriptive labels or categories (e.g., ["life", "inspiration"])

* The function merges them into one string for causal language modeling, allowing the model to learn contextual relationships between a quote and its tags.

2.Why str(example["tags"])

* tags is typically stored as a list (e.g., ["motivation", "success"]).

* Converting it to a string ensures it can be concatenated with the text.

3.map() Function

* The datasets.map() method applies the transformation to all examples efficiently using multiprocessing if available.

* It creates a new column "prediction" without altering the original data fields.

4.Output Example

"Life is beautiful. ->: ['life', 'inspiration']"




In [9]:
data['train'][0]

{'quote': '“Be yourself; everyone else is already taken.”',
 'author': 'Oscar Wilde',
 'tags': ['be-yourself',
  'gilbert-perreira',
  'honesty',
  'inspirational',
  'misattributed-oscar-wilde',
  'quote-investigator'],
 'prediction': "“Be yourself; everyone else is already taken.” ->: ['be-yourself', 'gilbert-perreira', 'honesty', 'inspirational', 'misattributed-oscar-wilde', 'quote-investigator']"}

In [10]:
# Tokenize the dataset for model input
# The tokenizer converts text into numerical token IDs that the model can process.
# Using `batched=True` allows processing multiple samples at once for efficiency.
data = data.map(
    lambda samples: tokenizer(samples['prediction']),
    batched=True
)


Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [11]:
data

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags', 'prediction', 'input_ids', 'attention_mask'],
        num_rows: 2508
    })
})

## LLM Fine Tuning

In [12]:
# Initialize the Hugging Face Trainer API for fine-tuning the model
# The Trainer simplifies the entire training loop, handling optimization, logging, and checkpointing.
trainer = transformers.Trainer(
    model=model,                          # LoRA-augmented model to be fine-tuned
    train_dataset=data['train'],          # The preprocessed and tokenized training dataset

    # Define training hyperparameters and runtime configurations
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,    # Reduced batch size to save memory
        gradient_accumulation_steps=4,    # Increased accumulation steps to compensate for smaller batch size
                                           # (Effective batch size = 4 x 4 = 16)
        warmup_steps=100,                 # Gradual learning rate warmup to stabilize early training
        max_steps=150,                    # Total number of optimization steps
        learning_rate=2e-4,               # Initial learning rate for the optimizer
        fp16=True,                        # Enables half-precision (float16) training for faster computation and lower memory use
        logging_steps=1,                  # Log training metrics after every step
        output_dir='outputs'              # Directory where checkpoints and logs will be saved
    ),

    # Define how batches are formed during training
    # DataCollatorForLanguageModeling dynamically pads sequences and prepares input/target pairs for causal LM
    data_collator=transformers.DataCollatorForLanguageModeling(
        tokenizer,
        mlm=False                          # Set to False for causal language modeling (predict next token)
    )
)

# Disable the model's cache during training to avoid warnings and memory inefficiency
# The cache is useful during inference but unnecessary for training
model.config.use_cache = False

# Begin fine-tuning the model
# The Trainer handles forward passes, loss computation, backward passes, and optimizer steps automatically
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.


Step,Training Loss
1,3.755600
2,3.602800
3,3.988100
4,3.369400
5,3.421700
6,3.096600
7,3.648600
8,3.642300
9,3.093400
10,3.366900


TrainOutput(global_step=150, training_loss=2.7193422134717307, metrics={'train_runtime': 148.975, 'train_samples_per_second': 8.055, 'train_steps_per_second': 1.007, 'total_flos': 686671030222848.0, 'train_loss': 2.7193422134717307, 'epoch': 0.4784688995215311})

## Share adapters on the 🤗 Hub

In [13]:
model.push_to_hub("firastlili/bloom-1b7-lora-finetuned",
                  use_auth_token=True,
                  commit_message="basic training",
                  private=True)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   4%|4         |  565kB / 12.6MB            

CommitInfo(commit_url='https://huggingface.co/firastlili/bloom-1b7-lora-finetuned/commit/84b535de478e16f0e49981ab3802c1e9f34deb88', commit_message='basic training', commit_description='', oid='84b535de478e16f0e49981ab3802c1e9f34deb88', pr_url=None, repo_url=RepoUrl('https://huggingface.co/firastlili/bloom-1b7-lora-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='firastlili/bloom-1b7-lora-finetuned'), pr_revision=None, pr_num=None)

## Load adapters from the Huggingface Hub

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
from transformers import BitsAndBytesConfig

# PEFT LoRA model ID
peft_model_id = "firastlili/bloom-1b7-lora-finetuned"

# Load PEFT config
config = PeftConfig.from_pretrained(peft_model_id)

# Define BitsAndBytes 8-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,        # recommended threshold
    llm_int8_enable_fp32_cpu_offload=False
)

# Load base BLOOM model with quantization via 'quantization_config'
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    device_map="auto",
    quantization_config=bnb_config,  # <-- use this instead of load_in_8bit
    return_dict=True
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load LoRA adapter on top of the base model
model = PeftModel.from_pretrained(model, peft_model_id)

print("✅ LoRA adapter loaded successfully with 8-bit quantization!")


adapter_config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/12.6M [00:00<?, ?B/s]

✅ LoRA adapter loaded successfully with 8-bit quantization!


🧠 Explanation

* PeftConfig.from_pretrained()
Loads the adapter configuration — includes info like LoRA rank r, target modules, scaling factors, and the base model name (config.base_model_name_or_path).

* AutoModelForCausalLM.from_pretrained()
Loads the original BLOOM model (not fine-tuned yet).
The load_in_8bit=True flag (if supported) enables efficient quantized loading using bitsandbytes.

* PeftModel.from_pretrained()
Applies the LoRA adapter weights from the specified PEFT model directory (e.g., "samwit/bloom-7b1-lora-tagger").
This effectively merges LoRA’s low-rank updates with the frozen base model.

* Result:
The model now behaves like a fine-tuned BLOOM model, but only a few million LoRA parameters have been modified — the rest remain frozen, enabling efficient inference.

## Model Inference

In [16]:
# Prepare the input text for the model
# The "->: " suffix is consistent with your training format: "<quote> ->: <tags>"
batch = tokenizer(
    "A room without books is like a body without a soul ->: ",  # input prompt
    return_tensors='pt'  # return PyTorch tensors
)

# Move input tensors to GPU if available
batch = {k: v.to(model.device) for k, v in batch.items()}

# Generate text using mixed-precision for efficiency
# torch.cuda.amp.autocast reduces memory usage and speeds up generation
with torch.cuda.amp.autocast():
    output_tokens = model.generate(
        **batch,
        max_new_tokens=50,       # maximum number of tokens to generate
        temperature=0.7,         # randomness: 0 = deterministic, higher = more creative
        top_p=0.9,               # nucleus sampling probability
        repetition_penalty=1.2   # penalize repeated tokens
    )

# Decode generated tokens into readable text
# skip_special_tokens=True removes tokens like <pad> or <eos>
generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

print("✅ Generated text:\n")
print(generated_text)


/tmp/ipython-input-3280317431.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


✅ Generated text:

A room without books is like a body without a soul ->:  reading, writing
Reading and Writing are two sides of the same coin. They both require effort to be done well.

A:

I think that there can only ever exist one right answer for this question.
The best way I know how would be: read


Explanation

1)Tokenization

* Converts raw text into input IDs the model understands.

* The "->: " suffix is used to mimic training format, helping the LoRA adapter predict tags or continue text appropriately.

2).to(model.device)

* Ensures the input tensors are on the same device as the model (GPU).

3)torch.cuda.amp.autocast()

* Uses automatic mixed precision (AMP) for faster generation and reduced GPU memory usage.

* Safe with LoRA adapters and 8-bit/4-bit quantized models.

4)model.generate() Parameters

* max_new_tokens=50: limits output length.

* temperature=0.7: controls creativity. Lower = more deterministic.

* top_p=0.9: nucleus sampling, keeps only the top 90% cumulative probability.

* repetition_penalty=1.2: discourages repeated phrases.

5)Decoding

* Converts token IDs back to human-readable text.

* skip_special_tokens=True removes <pad>, <eos>, etc.